# Dowload lyrics

In [ ]:
#Assign your Genius.com credentials and select your artist
import lyricsgenius as genius
token = "CRG5jc-SgTRDpuJzU2neM_SajlResplv7DMi9jn1LmlNA-WiTvn2XseA_4rWb6y8"
artist_name = "JuL"
geniusCreds = f"{token}"
artist_name = f"{artist_name}"

In [ ]:
Genius = genius.Genius(geniusCreds)
artist = Genius.search_artist(artist_name)

In [ ]:
artist.save_lyrics(ensure_ascii=True)

In [ ]:
import json
  
# Opening JSON file
f = open(f"Lyrics_{artist_name}_200.json")
  
# returns JSON object as 
# a dictionary
Artist = json.load(f)


In [ ]:
Artist['songs'][128]['lyrics'].split('\n\n')

In [ ]:
#Create an empty dictionary to store your songs and related data
artist_dict = {}
def collectSongData(adic):
    dps = list()
    title = adic['title'] #song title
    url = adic['url'] #spotify url
    artist = adic['artist'] #artist name(s)
    song_id = adic['id'] #spotify id
    lyrics = adic['lyrics'] #song lyrics
    upload_date = adic['description_annotation']['annotatable']['client_timestamps']['lyrics_updated_at'] #lyrics upload date
    annotations = adic['annotation_count'] #total no. of annotations
    descr = adic['description'] #song descriptions
    
    dps.append((title,url,artist,song_id,lyrics,upload_date,annotations,descr)) #append all to one tuple list
    artist_dict[title] = dps #assign list to song dictionary entry named after song title
    
for song in Artist['songs']: 
    collectSongData(song) #check function works

In [ ]:
import csv

def updateCSV_file():
    upload_count = 0 #Set upload counter
    location = f"./" #Pick file location
    print("input filename of song file, please add .csv")
    filename = input() #give your file a name
    file = location + filename
    with open(file, 'w', newline='', encoding='utf-8') as file: #open a new csv file
        a = csv.writer(file, delimiter=',') #split by comma
        #(title,url,artist,song_id,lyrics,year,upload_date,annotations,descr)
        headers = ["Title","URL","Artist", "Song ID", "Lyrics", "Upload Date", "Annotations","Description"] #create header row
        a.writerow(headers) #add header row
        for song in artist_dict:
            a.writerow(artist_dict[song][0])
            upload_count+=1
            
        print(str(upload_count) + " songs have been uploaded")
updateCSV_file()

# First model

## Clean data

In [2]:
df = pd.read_csv('./Lyrics_JuL_200.csv')

In [3]:
def split_text(text):
    text = text['Lyrics']
    keys = re.findall("\[(.*?)\]", text)[1:]
    for k in keys:
        text = text.replace(k, '@@')
    sections = text.split('@@')    
    
    pattern = re.findall(r'\d+Embed', sections[-1])
    if len(pattern) == 0:
        pattern = re.findall(r'Embed', sections[-1])
    if len(pattern) > 0:
        sections[-1] = sections[-1].replace(pattern[0],'')

    lyrics = str()

    single_text = []

    res = {}
    for s in sections:
        single_text += [x.lower().replace('(','').replace(')','').translate(translator) for x in s[s.find(']')+1:].split('\n') if len(x) > 1]

    res['single_text'] =  ' \n '.join(single_text)
    return pd.Series(res)

In [4]:
df = df.join(df.apply(split_text, axis=1))

# ML

In [1]:
import io
import os
import sys
import string
import numpy as np
import pandas as pd
from tensorflow import keras
from __future__ import print_function
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import LambdaCallback, ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Dense, Dropout, Activation, LSTM, Bidirectional, Embedding
translator = str.maketrans('', '', string.punctuation)
import re

In [ ]:
# Compute word frequencies
text_as_list = []

frequencies = {}

uncommon_words = set()

MIN_FREQUENCY = 7

MIN_SEQ = 5

BATCH_SIZE =  32


def extract_text(text):

    global text_as_list

    text_as_list += [w for w in text.split(' ') if w.strip() != '' or w == '\n']

df['single_text'].apply( extract_text )
print('Total words: ', len(text_as_list))


for w in text_as_list:
    frequencies[w] = frequencies.get(w, 0) + 1

uncommon_words = set([key for key in frequencies.keys() if frequencies[key] < MIN_FREQUENCY])
words = sorted(set([key for key in frequencies.keys() if frequencies[key] >= MIN_FREQUENCY]))


num_words = len(words)

word_indices = dict((w, i) for i, w in enumerate(words))

indices_word = dict((i, w) for i, w in enumerate(words))

print('Words with less than {} appearances: {}'.format( MIN_FREQUENCY, len(uncommon_words)))

print('Words with more than {} appearances: {}'.format( MIN_FREQUENCY, len(words)))


valid_seqs = []

end_seq_words = []

for i in range(len(text_as_list) - MIN_SEQ):

    end_slice = i + MIN_SEQ + 1

    if len(set(text_as_list[i:end_slice]).intersection(uncommon_words) ) == 0:

        valid_seqs.append(text_as_list[i: i + MIN_SEQ])

        end_seq_words.append(text_as_list[i + MIN_SEQ])


print('Valid sequences of size {}: {}'.format(MIN_SEQ, len(valid_seqs)))


X_train, X_test, y_train, y_test = train_test_split(valid_seqs, end_seq_words, test_size=0.02, random_state=42)

print(X_train[2:5])

In [ ]:
series = pd.Series(frequencies)

In [ ]:
plt.figure(dpi=200, figsize=(25, 10))
sdf = series.sort_values(ascending=False)[100:200]
plt.scatter(sdf.index, sdf)
plt.xticks(rotation = 45);

In [ ]:
# Data generator for fit and evaluate
def generator(sentence_list, next_word_list, batch_size):
    index = 0
    while True:
        x = np.zeros((batch_size, MIN_SEQ), dtype=np.int32)
        y = np.zeros((batch_size), dtype=np.int32)
        
        for i in range(batch_size):
            for t, w in enumerate(sentence_list[index % len(sentence_list)]):
                x[i, t] = word_indices[w]
            y[i] = word_indices[next_word_list[index % len(sentence_list)]]
            index = index + 1
        yield x, y


# Functions from keras-team/keras/blob/master/examples/lstm_text_generation.py
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)

    preds = exp_preds / np.sum(exp_preds)

    probas = np.random.multinomial(1, preds, 1)

    return np.argmax(probas)


def on_epoch_end(epoch, logs):

    # Function invoked at end of each epoch. Prints generated text.
    examples_file.write('\n----- Generating text after Epoch: %d\n' % epoch)


    # Randomly pick a seed sequence
    seed_index = np.random.randint(len(X_train+X_test))

    seed = (X_train+X_test)[seed_index]


    for diversity in [0.3, 0.4, 0.5, 0.6, 0.7]:
        sentence = seed

        examples_file.write('----- Diversity:' + str(diversity) + '\n')
        examples_file.write('----- Generating with seed:\n"' + ' '.join(sentence) + '"\n')

        examples_file.write(' '.join(sentence))


        for i in range(50):

            x_pred = np.zeros((1, MIN_SEQ))

            for t, word in enumerate(sentence):
                x_pred[0, t] = word_indices[word]


            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_word = indices_word[next_index]
            sentence = sentence[1:]
            sentence.append(next_word)
            examples_file.write(" "+next_word)

        examples_file.write('\n')

    examples_file.write('='*80 + '\n')

    examples_file.flush()

In [ ]:
def get_model():
    print('Build model...')
    model = Sequential()
    model.add(Embedding(input_dim=len(words), output_dim=1024))
    model.add(Bidirectional(LSTM(128)))
    model.add(Dense(len(words)))
    model.add(Activation('softmax'))
    return model

In [ ]:
model = get_model()

model.compile(loss='sparse_categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

file_path = "./checkpoints/LSTM_LYRICS-epoch{epoch:03d}-words%d-sequence%d-minfreq%d-" \
            "loss{loss:.4f}-acc{accuracy:.4f}-val_loss{val_loss:.4f}-val_acc{val_accuracy:.4f}" % \
            (len(words), MIN_SEQ, MIN_FREQUENCY)


checkpoint = ModelCheckpoint(file_path, monitor='val_accuracy', save_best_only=True)

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

early_stopping = EarlyStopping(monitor='val_accuracy', patience=20)

callbacks_list = [checkpoint, print_callback, early_stopping]


examples_file = open('examples.txt', "w")

model.fit(generator(X_train, y_train, BATCH_SIZE),
                    steps_per_epoch=int(len(valid_seqs) / BATCH_SIZE) + 1,
                    epochs=200,
                    callbacks=callbacks_list,
                    validation_data=generator(X_test, y_train, BATCH_SIZE),
                    validation_steps=int(len(y_train) / BATCH_SIZE) + 1)

# Seconde model

## Clean data

In [5]:
text = ''
for lyrics in df.single_text:
    text += '\n' + lyrics

In [6]:
corpus = text.lower().split(" \n ")

In [7]:
corpus = list(set(corpus))

In [8]:
tokenizer = keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

In [9]:
# create input sequences using list of tokens
input_sequences = []
for line in corpus[1:]:
    token_list = tokenizer.texts_to_sequences([line])[0]

    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]

    input_sequences.append(n_gram_sequence)

In [10]:
# pad sequences
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(keras.utils.pad_sequences(input_sequences,
                           maxlen = max_sequence_len, padding='pre'))

In [11]:
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]

117

## Training

In [23]:
model = Sequential()

model.add(Embedding(total_words, 50, 
                    input_length=max_sequence_len-1))

# Add an LSTM Layer
model.add(Bidirectional(LSTM(150, return_sequences=True)))  

# A dropout layer for regularisation
model.add(Dropout(0.2))

# Add another LSTM Layer
model.add(LSTM(100))

model.add(Dense(total_words / 2, activation='relu')) 

# In the last layer, the shape should be equal to the total number of words present in our corpus
model.add(Dense(total_words, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy']) 

#(# Pick a loss function and an optimizer)
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 116, 50)           547200    
                                                                 
 bidirectional_4 (Bidirectio  (None, 116, 300)         241200    
 nal)                                                            
                                                                 
 dropout_3 (Dropout)         (None, 116, 300)          0         
                                                                 
 lstm_8 (LSTM)               (None, 100)               160400    
                                                                 
 dense_6 (Dense)             (None, 5472)              552672    
                                                                 
 dense_7 (Dense)             (None, 10944)             59896512  
                                                      

In [ ]:
history = model.fit(predictors, label, epochs= 100, verbose=1)

Epoch 1/100
311/311 [==============================] - 166s 519ms/step - loss: 8.8973 - accuracy: 0.0173
Epoch 2/100
311/311 [==============================] - 172s 553ms/step - loss: 8.1372 - accuracy: 0.0178
Epoch 3/100
311/311 [==============================] - 189s 607ms/step - loss: 7.6840 - accuracy: 0.0183
Epoch 4/100
311/311 [==============================] - 184s 590ms/step - loss: 7.0908 - accuracy: 0.0226
Epoch 5/100
311/311 [==============================] - 192s 618ms/step - loss: 6.2435 - accuracy: 0.0417
Epoch 6/100
311/311 [==============================] - 191s 613ms/step - loss: 5.1545 - accuracy: 0.0783
Epoch 7/100
311/311 [==============================] - 205s 659ms/step - loss: 3.9049 - accuracy: 0.1811
Epoch 8/100
311/311 [==============================] - 193s 620ms/step - loss: 2.6735 - accuracy: 0.3607
Epoch 9/100
311/311 [==============================] - 178s 571ms/step - loss: 1.7403 - accuracy: 0.5579
Epoch 10/100
311/311 [==============================] -

In [ ]:
model.save('./first_model')


In [ ]:
def make_lyrics(seed_text, next_words):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list],
                     maxlen=max_sequence_len-1,padding='pre')        predicted = model.predict_classes(token_list, verbose=0)        output_word = ""        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    print(seed_text)